In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Thư viện :

In [2]:
!pip install torch onnx safetensors diffusers transformers
# !pip install torch onnx safetensors

In [3]:
from huggingface_hub import snapshot_download

# Đường dẫn thư mục lưu trên Google Drive
save_path = "text_encoder"

# Tải toàn bộ thư mục `unet`
snapshot_download(
    repo_id="timbrooks/instruct-pix2pix",
    local_dir=save_path,
    allow_patterns=["text_encoder/*"]  # Chỉ tải thư mục `unet`
)

print(f"✅ Đã tải xong thư mục `text_encoder` vào {save_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/492M [00:00<?, ?B/s]

text_encoder%2Fconfig.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

pytorch_model.fp16.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

✅ Đã tải xong thư mục `text_encoder` vào text_encoder


In [4]:
!pip install  accelerate

In [1]:
import torch
import os
import json
from transformers import CLIPTextModel
from safetensors.torch import load_file
from transformers import CLIPTextConfig

# 🔹 Đường dẫn tới model và config
config_path = "/content/text_encoder/text_encoder/config.json"
safetensor_path = "/content/text_encoder/text_encoder/model.fp16.safetensors"

# 🔹 Nơi lưu file ONNX
onnx_dir = "/content/drive/MyDrive/Model/text_encoder/model_onnx"
os.makedirs(onnx_dir, exist_ok=True)
onnx_path = os.path.join(onnx_dir, "text_encoder.onnx")

# 🔹 Load config từ file JSON
with open(config_path, "r") as f:
    config = json.load(f)

# 🔹 Tạo mô hình CLIPTextModel từ config
text_config = CLIPTextConfig(**config)
text_encoder = CLIPTextModel(text_config)

# 🔹 Load trọng số từ file safetensors
state_dict = load_file(safetensor_path)
text_encoder.load_state_dict(state_dict, strict=False)  # 🔹 strict=False để tránh lỗi thiếu key

# 🔹 Chuyển mô hình sang chế độ đánh giá
text_encoder.eval()

# 🔹 Tạo tensor input giả lập
batch_size = 1
seq_length = config["max_position_embeddings"]
hidden_size = config["hidden_size"]

dummy_input = torch.randint(0, config["vocab_size"], (batch_size, seq_length), dtype=torch.int64)  # Token IDs

# 🔹 Tạo Wrapper để đưa vào ONNX
class TextEncoderWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids):
        return self.model(input_ids).last_hidden_state  # Lấy hidden state cuối

wrapped_text_encoder = TextEncoderWrapper(text_encoder)

# 🔹 Xuất mô hình sang ONNX
torch.onnx.export(
    wrapped_text_encoder,
    (dummy_input,),
    onnx_path,
    export_params=True,
    opset_version=17,  # 🔹 Sử dụng ONNX >= 17
    do_constant_folding=True,
    input_names=["input_ids"],
    output_names=["last_hidden_state"],
    dynamic_axes={"input_ids": {0: "batch_size"}, "last_hidden_state": {0: "batch_size"}}
)

print(f"✅ Convert thành công! Mô hình ONNX đã lưu tại: {onnx_path}")


/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py:88: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py:164: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:


✅ Convert thành công! Mô hình ONNX đã lưu tại: /content/drive/MyDrive/Model/text_encoder/model_onnx/text_encoder.onnx


In [2]:
import os

onnx_path = "/content/drive/MyDrive/Model/text_encoder/model_onnx/text_encoder_16.onnx"
print("ONNX File Exists:", os.path.exists(onnx_path))


ONNX File Exists: True


In [3]:
!pip install onnxruntime

In [4]:
import onnxruntime as ort

onnx_path = "/content/drive/MyDrive/Model/text_encoder/model_onnx/text_encoder_16.onnx"

# Load ONNX bằng onnxruntime (ít tốn RAM hơn)
session = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
print("🚀 ONNX model đã load thành công!")


🚀 ONNX model đã load thành công!
